In [1]:
# Seed for reproducibility
import torch
import numpy as np
import pandas as pd
import scanpy as sc
from typing import Tuple

import sys
sys.path.append('..')
from tools.formating.formating import *
from tools.utils.utils import run_dimension_reduction
from tools.evaluation.monitor import *

# scVI imports
import scvi
print(scvi.__version__)
from scvi.model.utils import mde
import pymde

torch.manual_seed(0)
np.random.seed(0)
sc.settings.verbosity = 0  # verbosity: errors (0), warnings (1), info (2), hints (3)

/usr/local/lib/python3.8/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.8/dist-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda9SetDeviceEi'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
Global seed set to 0


0.20.3


In [2]:
input = "/ps/ai-ready/data/Clustering/Tabula_Sapiens/TS_Skin_test.h5ad"
adata = load_anndata(input)
adata

/ps/ai-ready/data/Clustering/Tabula_Sapiens/TS_Skin_test.h5ad


AnnData object with n_obs × n_vars = 725 × 3853
    obs: 'organ_tissue', 'method', 'donor', 'anatomical_information', 'n_counts_UMIs', 'n_genes', 'cell_ontology_class', 'free_annotation', 'manually_annotated', 'compartment', 'gender', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'doublet_scores', 'predicted_doublets'
    var: 'gene_symbol', 'feature_type', 'ensemblid', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std', 'n_cells', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: '_scvi', '_training_mode', 'cell_ontology_class_colors', 'dendrogram_cell_type_tissue', 'dendrogram_comput

In [3]:
monitor = Monitor(1)

In [4]:
# scVI uses non normalized data so we keep the original data in a separate AnnData object, then the normalization steps are performed
scvi.model.SCVI.setup_anndata(
    adata,
    layer="raw_counts"
)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [5]:
model = scvi.model.SCVI(adata)
model

SCVI Model with the following params: 
n_hidden: 128, n_latent: 10, n_layers: 1, dropout_rate: 0.1, dispersion: gene, gene_likelihood: zinb, 
latent_distribution: normal
Training status: Not Trained
Model's adata is minified?: False

In [6]:
model.train()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/trainer.py:1609: PossibleUserWarning: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 400/400: 100%|█████████████████████████████████████████████████████████████████| 400/400 [01:03<00:00,  6.53it/s, loss=1.28e+03, v_num=1]

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|█████████████████████████████████████████████████████████████████| 400/400 [01:03<00:00,  6.30it/s, loss=1.28e+03, v_num=1]


In [7]:
latent = model.get_latent_representation()
adata.obsm["X_scVI"] = latent

In [8]:
denoised = model.get_normalized_expression(adata, library_size=1e4)
adata.layers["scvi_normalized"] = model.get_normalized_expression(
    library_size=10e4
)

In [9]:
adata.layers["raw_counts"].toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [10]:
adata.obsm["X_scVI"].shape[1]

10

In [11]:
adata = run_dimension_reduction(adata, use_rep='X_scVI', n_pcs=10, n_neighbors=15)

In [12]:
adata.obsm["X_mde"] = mde(adata.obsm["X_scVI"])

In [13]:
time_points, cpu_usage, mem_usage, gpu_mem_usage = monitor.stop()

In [14]:
time_points

[1701177583.4549053,
 1701177584.4593751,
 1701177585.4616423,
 1701177586.4636798,
 1701177587.46596,
 1701177588.4720194,
 1701177589.474258,
 1701177590.4765682,
 1701177591.4788644,
 1701177592.4813144,
 1701177593.4845297,
 1701177594.487297,
 1701177595.4899368,
 1701177596.4929013,
 1701177597.4973128,
 1701177598.5002606,
 1701177599.5025635,
 1701177600.5058699,
 1701177601.5079699,
 1701177602.5110228,
 1701177603.5132551,
 1701177604.515334,
 1701177605.517458,
 1701177606.5205548,
 1701177607.522703,
 1701177608.5248132,
 1701177609.527099,
 1701177610.5291414,
 1701177611.5323246,
 1701177612.5347111,
 1701177613.5375285,
 1701177614.5396166,
 1701177615.5417125,
 1701177616.5448263,
 1701177617.546865,
 1701177618.5492244,
 1701177619.5531325,
 1701177620.5552733,
 1701177621.5574887,
 1701177622.5595796,
 1701177623.5615797,
 1701177624.563613,
 1701177625.5657089,
 1701177626.5687134,
 1701177627.5708015,
 1701177628.572834,
 1701177629.5755126,
 1701177630.577932,
 170

In [15]:
x = [n for n in range(len(time_points))]
x

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86]

In [16]:
cpu_usage

[0.0,
 30.5,
 43.8,
 44.1,
 42.6,
 43.1,
 43.1,
 42.0,
 41.5,
 41.2,
 42.2,
 41.9,
 41.6,
 41.0,
 42.6,
 42.1,
 41.6,
 42.3,
 41.1,
 42.0,
 42.2,
 43.0,
 41.6,
 41.8,
 42.1,
 42.4,
 41.7,
 41.2,
 41.8,
 40.9,
 41.7,
 41.2,
 42.6,
 42.4,
 43.2,
 42.5,
 42.7,
 42.1,
 41.3,
 41.9,
 42.2,
 42.3,
 41.5,
 40.2,
 42.3,
 41.4,
 41.0,
 41.4,
 42.1,
 42.6,
 42.3,
 42.2,
 41.8,
 41.4,
 42.5,
 41.5,
 42.5,
 42.5,
 42.7,
 41.1,
 41.8,
 42.0,
 42.8,
 40.9,
 27.1,
 57.0,
 82.0,
 95.2,
 100.0,
 41.8,
 2.4,
 1.8,
 1.9,
 1.8,
 1.8,
 1.9,
 1.8,
 1.8,
 10.9,
 58.5,
 50.1,
 21.6,
 1.8,
 27.6,
 76.8,
 87.0,
 89.8]

In [17]:
mem_usage

[1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6,
 1.6]

In [18]:
# gpu_mem_usage

In [19]:
sum(gpu_mem_usage) == 0

True

In [20]:
adata.obs

,organ_tissue,method,donor,anatomical_information,n_counts_UMIs,n_genes,cell_ontology_class,free_annotation,manually_annotated,compartment,...,log1p_total_counts_hb,pct_counts_hb,outlier,mt_outlier,doublet_scores,predicted_doublets,_scvi_batch,_scvi_labels,leiden_X_scVI,louvain_X_scVI
cell_id,,,,,,,,,,,,,,,,,,,,,
AAACCCAAGTTATGGA_TSP10_Skin_NA_10X_1_1,Skin,10X,TSP10,nan,2695.0,890,"cd8-positive, alpha-beta memory t cell",CD8-positive memory T cell,True,immune,...,0.693147,0.036900,False,False,0.050685,False,0,0,2,1
AACAAGACAGCAGGAT_TSP10_Skin_NA_10X_1_1,Skin,10X,TSP10,nan,3207.0,1033,"cd4-positive, alpha-beta memory t cell",CD4-positive memory T cell,True,immune,...,0.693147,0.030998,False,False,0.070626,False,0,0,2,1
AAGATAGAGCCATATC_TSP10_Skin_NA_10X_1_1,Skin,10X,TSP10,nan,3564.0,1148,"naive thymus-derived cd8-positive, alpha-beta ...",Naive CD8-positive T cell,True,immune,...,0.000000,0.000000,False,False,0.060601,False,0,0,4,2
AAGGTAACAATCCTAG_TSP10_Skin_NA_10X_1_1,Skin,10X,TSP10,nan,4133.0,1187,t cell,alpha-beta T cell,True,immune,...,0.693147,0.024027,False,False,0.024758,False,0,0,2,1
AAGTCGTAGCAGTAAT_TSP10_Skin_NA_10X_1_1,Skin,10X,TSP10,nan,3731.0,1737,stromal cell,stromal cells,True,stromal,...,0.000000,0.000000,False,False,0.357937,True,0,0,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGATGGAGTGGCGAT_TSP14_Skin_Chest_10X_1_1,Skin,10X,TSP14,Chest,4114.0,1490,t cell,alpha-beta T cell,True,immune,...,0.000000,0.000000,False,False,0.058805,False,0,0,9,8
TTGCCTGGTTACCCTC_TSP14_Skin_Chest_10X_1_1,Skin,10X,TSP14,Chest,16208.0,3578,stromal cell,stromal cells,True,stromal,...,0.000000,0.000000,False,False,0.027185,False,0,0,3,0
TTGTGGACACAGTGTT_TSP14_Skin_Chest_10X_1_1,Skin,10X,TSP14,Chest,25516.0,4128,endothelial cell,endothelial cells,True,endothelial,...,1.791759,0.019158,False,False,0.043766,False,0,0,1,3


In [21]:
from sklearn.metrics.cluster import adjusted_rand_score
ari=adjusted_rand_score(adata.obs["cell_ontology_class"], adata.obs["leiden_X_scVI"])
ari

0.43429292819059256

In [22]:
adata.obs["cell_ontology_class"].nunique()

22

In [23]:
# from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score as ARI
from sklearn.metrics import normalized_mutual_info_score as NMI
from sklearn.metrics import silhouette_score
# from sklearn.mixture import GaussianMixture as GMM

def clustering_scores(labels, labels_pred, embedding):
    asw_score = silhouette_score(embedding, labels)
    nmi_score = NMI(labels, labels_pred)
    ari_score = ARI(labels, labels_pred)
    print(
        "Clustering Scores:\nSilhouette: %.4f\nNMI: %.4f\nARI: %.4f"
        % (asw_score, nmi_score, ari_score)
    )
    return asw_score, nmi_score, ari_score

In [24]:
asw_score, nmi_score, ari_score = clustering_scores(adata.obs["cell_ontology_class"], adata.obs["leiden_X_scVI"], adata.obsm["X_mde"])

Clustering Scores:
Silhouette: 0.2348
NMI: 0.6572
ARI: 0.4343


In [25]:
from datetime import datetime

datetime.fromtimestamp(1701102855.5850654)

datetime.datetime(2023, 11, 27, 16, 34, 15, 585065)

In [26]:
y = dict(CPU=cpu_usage, Memory=mem_usage, GPU=gpu_mem_usage)

In [27]:
type(y)

dict

In [28]:
from tools.visualization.plot import *

line = plot_line(time_points, y)

with open('line.txt','w') as f:
   f.write(str(line))

line

'{"data": [{"type": "scatter", "x": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86], "y": [0.0, 30.5, 43.8, 44.1, 42.6, 43.1, 43.1, 42.0, 41.5, 41.2, 42.2, 41.9, 41.6, 41.0, 42.6, 42.1, 41.6, 42.3, 41.1, 42.0, 42.2, 43.0, 41.6, 41.8, 42.1, 42.4, 41.7, 41.2, 41.8, 40.9, 41.7, 41.2, 42.6, 42.4, 43.2, 42.5, 42.7, 42.1, 41.3, 41.9, 42.2, 42.3, 41.5, 40.2, 42.3, 41.4, 41.0, 41.4, 42.1, 42.6, 42.3, 42.2, 41.8, 41.4, 42.5, 41.5, 42.5, 42.5, 42.7, 41.1, 41.8, 42.0, 42.8, 40.9, 27.1, 57.0, 82.0, 95.2, 100.0, 41.8, 2.4, 1.8, 1.9, 1.8, 1.8, 1.9, 1.8, 1.8, 10.9, 58.5, 50.1, 21.6, 1.8, 27.6, 76.8, 87.0, 89.8], "name": "CPU", "mode": "lines+markers", "line": {"dash": "solid"}, "marker": {"size": 7}, "connectgaps":

In [29]:
x= ['ARI', 'Silhouette', 'NMI']
y1 = dict(scVI=[ari_score, asw_score, nmi_score])

In [30]:
from tools.visualization.plot import *

bar = plot_bar(x, y1)

with open('bar.txt','w') as f:
   f.write(str(bar))

bar

'{"data": [{"type": "bar", "x": ["ARI", "Silhouette", "NMI"], "y": [0.4343, 0.2348, 0.6572], "text": [0.4343, 0.2348, 0.6572], "textposition": "auto", "name": "scVI", "marker": {"opacity": 0.5}}], "layout": {"title": "Benchmarks", "xaxis": {"tickangle": 0}, "margin": {"r": 50, "l": 50, "t": 50, "b": 50}, "barmode": "group", "hovermode": "closest", "transition": {"duration": 100}, "autosize": true, "width": 1000, "height": 750}}'